### Check Spatial Registration to Allen V3

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import nrrd
import os

import anndata

from dredFISH.Utils import regu

# import importlib
# importlib.reload(regu)

In [2]:
%%time
# allen data
allen_template_path= '/greendata/binfo/mouse/Brain/average_template.npy'
allen_tree_path    = '/greendata/GeneralStorage/fangming/reference/allen_ccf/structures.json'
allen_annot_path   = '/greendata/GeneralStorage/fangming/reference/allen_ccf/annotation_10.nrrd'

allen_template = regu.load_allen_template(allen_template_path)
allen_tree, allen_maps = regu.load_allen_tree(allen_tree_path)
allen_annot = regu.load_allen_annot(allen_annot_path) # takes about 30 seconds

CPU times: user 14.1 s, sys: 10.3 s, total: 24.4 s
Wall time: 24.4 s


In [3]:
# # plot alen average template front -> back
# rows = 10
# columns = 10
# fig, axs = plt.subplots(columns,rows,figsize=[25,22],)
# nbins = columns*rows
# bins = np.linspace(10,allen_template.shape[0]-10,nbins).astype(int)
# i = 0
# for ax in axs.ravel():
#     bin = bins[i]
#     img = allen_template[bin,:,:]
#     if img.max()==0:
#         continue
#     vmin,vmax=np.percentile(img[img>0].ravel(),[5,95])
#     ax.imshow(img,vmin=vmin,vmax=vmax,cmap='jet')
#     ax.set_title(bin)
#     ax.set_axis_off()
#     i+=1
# plt.subplots_adjust(wspace=0, 
#                     hspace=0.1)

In [5]:
# dredFISH data input
dataset   = 'DPNMF_PolyA_2021Nov19' # dataset tag
data_path = f'/greendata/Images2021/gaby/dredFISH/DPNMF_PolyA_2021Nov19/results/raw_fishdata_Mar30.h5ad'

# ## output
version = 'v3'
outdir  = '/greendata/GeneralStorage/fangming/projects/dredfish/data_dump/'
reg_output_prefix = os.path.join(outdir, f"{dataset}_{version}_spatialreg_")
print(reg_output_prefix)
reg_force_overwrite = True

/greendata/GeneralStorage/fangming/projects/dredfish/data_dump/DPNMF_PolyA_2021Nov19_v3_spatialreg_


In [ ]:
df = anndata.read(data_path, backed='r').obs
XY = df[['stage_x', 'stage_y']].values
XY

In [ ]:
registration_idx  = 720 # 916
registration_flip = False # True
regu.check_run(XY, 
                allen_template, 
                allen_annot, 
                allen_maps,
                registration_idx, 
                flip=registration_flip)

In [ ]:
spatial_data = regu.real_run(XY, 
                            allen_template, 
                            allen_annot, 
                            allen_maps,
                            registration_idx, 
                            flip=registration_flip,
                            name="", 
                            outprefix=registration_output_prefix, # CHECK THIS
                            force=registration_force_overwrite,
                            )
# update results to anndata (cell level atrributes)
df_res = pd.DataFrame()
df_res['coord_x']        = np.array(spatial_data.points_rot[:,0])
df_res['coord_y']        = np.array(spatial_data.points_rot[:,1])
df_res['region_id']      = np.array(spatial_data.region_id)
df_res['region_color']   = np.array(spatial_data.region_color) 
df_res['region_acronym'] = np.array(spatial_data.region_acronym)
# self.save_data()

In [ ]:
df_res

In [ ]:
# check results -- plot all cells
def plot_spatial(df_res):
    """
    """
    fig, ax = plt.subplots(1, 1, figsize=(8,6))
    ax.scatter(df_res['coord_x'], 
               df_res['coord_y'],
               c=df_res['region_color'],
               s=0.05,
               rasterized=True,
              )
    ax.invert_yaxis()
    ax.set_aspect('equal')
    ax.set_title('Cells colored by brain regions (moved ABA)')
    ax.axis('off')
    plt.show()

In [ ]:
plot_spatial(df_res)